In [1]:
import ROOT
ROOT.RooMsgService.instance().setGlobalKillBelow(5)

Welcome to JupyROOT 6.11/01


In [2]:
meas = ROOT.RooStats.HistFactory.Measurement( "meas", "meas" )
meas.SetPOI( "SignalStrength" )
meas.SetLumi( 1.0 )
meas.SetLumiRelErr( 0.02 )
meas.AddConstantParam( "Lumi" )

## Make some example data

expected and observed data, one bin, 10% more events observed than expected so SignalStrength should be 1.1

In [3]:
data_hist = ROOT.TH1D("observed","observed",1,0,1)
for i in range(1100):
    data_hist.Fill(0.5)
signal_hist = ROOT.TH1D("above_expected","above_expected",1,0,1)
for i in range(100):
    signal_hist.Fill(0.5)
model_hist = ROOT.TH1D("expected","expected",1,0,1)
for i in range(1000):
    model_hist.Fill(0.5)

## Create a measurement  and fill it.

In [4]:
chan = ROOT.RooStats.HistFactory.Channel( "Region1" )
chan.SetStatErrorConfig(0.05, "Poisson")
chan.SetData( data_hist )

In [5]:
model = ROOT.RooStats.HistFactory.Sample( "model" )
model.SetNormalizeByTheory( False )
model.SetHisto( model_hist )

signal = ROOT.RooStats.HistFactory.Sample( "signal" )
signal.SetNormalizeByTheory( False )
signal.SetHisto( signal_hist )

And add our parameter of interest with a sensible bound.

In [6]:
signal.AddNormFactor( "SignalStrength", 1, 0, 3)

and one nuisance parameter

In [7]:
uncertainty_up   = 1000 * 1.1
uncertainty_down = 1000 * 0.9
signal.AddOverallSys( "signal_norm_uncertainty",  uncertainty_down*.1, uncertainty_up*.1 )
model.AddOverallSys( "background_norm_uncertainty",  uncertainty_down,uncertainty_up )

In [8]:
sig_np_up = signal_hist.Clone()
sig_np_down = signal_hist.Clone()
bkg_np_up = model_hist.Clone()
bkg_np_down = model_hist.Clone()
for b in range(1,sig_np_up.GetNbinsX()+1):
    sig_np_up.SetBinContent(b, sig_np_up.GetBinContent(b) + sig_np_up.GetBinContent(b) * .1 * b)
    sig_np_down.SetBinContent(b, sig_np_down.GetBinContent(b) - sig_np_down.GetBinContent(b) * 0.1 * b)
    bkg_np_up.SetBinContent(b, bkg_np_up.GetBinContent(b) + bkg_np_up.GetBinContent(b) * .1 * b)
    bkg_np_down.SetBinContent(b, bkg_np_down.GetBinContent(b) - bkg_np_down.GetBinContent(b) * 0.1 * b)

In [9]:
signal_shape = ROOT.RooStats.HistFactory.HistoSys("signal_shape")
signal_shape.SetHistoHigh( sig_np_up )
signal_shape.SetHistoLow( sig_np_down )
signal.AddHistoSys( signal_shape )

background_shape = ROOT.RooStats.HistFactory.HistoSys("background_shape")
background_shape.SetHistoHigh( bkg_np_up )
background_shape.SetHistoLow( bkg_np_down )
model.AddHistoSys( background_shape )

And add to measuremnet

In [10]:
chan.AddSample( model )
chan.AddSample( signal )
meas.AddChannel( chan )

## Make workspace!

In [11]:
hist2workspace = ROOT.RooStats.HistFactory.HistoToWorkspaceFactoryFast(meas)

In [12]:
workspace = hist2workspace.MakeSingleChannelModel( meas, chan )



-------------------
Starting to process Region1 channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,1.2],0.02
Gaussian::alpha_background_shapeConstraint(alpha_background_shape,nom_alpha_background_shape[0.,-10,10],1.)
making normFactor: SignalStrength
Gaussian::alpha_signal_shapeConstraint(alpha_signal_shape,nom_alpha_signal_shape[0.,-10,10],1.)
-----------------------------------------
import model into workspace
RooDataSet::AsimovData[obs_x_Region1,weight:binWeightAsimov] = 1 entries (1100 weighted)

RooWorkspace(Region1) Region1 workspace contents

variables
---------
(Lumi,SignalStrength,alpha_background_norm_uncertainty,alpha_background_shape,alpha_signal_norm_uncertainty,alpha_signal_shape,binWidth_obs_x_Region1_0,binWidth_obs_x_Region1_1,nom_alpha_background_norm_uncertainty,nom_alpha_background_shape,nom_alpha_signal_norm_uncertainty,nom_alpha_signal_shape,nominalLumi,obs_x_Region1,weightVar)

p.d.f.s
-------
RooRealSumPdf::Region1_model[ binWidth_

ok this was put into a function...

In [13]:
from Builder import get_workspace

In [14]:
workspace = get_workspace(nchannels = 1, events = 1000, nbins = 1)



-------------------
Starting to process Region0 channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,1.2],0.02
model_Region0 has no variation histograms 
processing hist expected0
making normFactor: SignalStrength
signal_Region0 has no variation histograms 
processing hist above_expected0
-----------------------------------------
import model into workspace
RooDataSet::AsimovData[obs_x_Region0,weight:binWeightAsimov] = 1 entries (990 weighted)

RooWorkspace(Region0) Region0 workspace contents

variables
---------
(Lumi,SignalStrength,binWidth_obs_x_Region0_0,binWidth_obs_x_Region0_1,nominalLumi,obs_x_Region0,weightVar)

p.d.f.s
-------
RooRealSumPdf::Region0_model[ binWidth_obs_x_Region0_0 * L_x_model_Region0_overallSyst_x_Exp + binWidth_obs_x_Region0_1 * L_x_signal_Region0_overallSyst_x_Exp ] = 990/990
RooGaussian::lumiConstraint[ x=Lumi mean=nominalLumi sigma=0.02 ] = 1
RooProdPdf::model_Region0[ lumiConstraint * Region0_model(obs_x_Region0) ] = 990

func

ok that seemed to work

In [15]:
workspace.SetName('BinnedWorkspace')
workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(1,1000,1,0))

False

In [16]:
events = 1000
chans = 1
nps = 0
for bins in [1,10,20,30,40,50,60,70,80,90,100]:
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))



-------------------
Starting to process Region0 channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,1.2],0.02
model_Region0 has no variation histograms 
processing hist expected0
making normFactor: SignalStrength
signal_Region0 has no variation histograms 
processing hist above_expected0
-----------------------------------------
import model into workspace
RooDataSet::AsimovData[obs_x_Region0,weight:binWeightAsimov] = 1 entries (990 weighted)

RooWorkspace(Region0) Region0 workspace contents

variables
---------
(Lumi,SignalStrength,binWidth_obs_x_Region0_0,binWidth_obs_x_Region0_1,nominalLumi,obs_x_Region0,weightVar)

p.d.f.s
-------
RooRealSumPdf::Region0_model[ binWidth_obs_x_Region0_0 * L_x_model_Region0_overallSyst_x_Exp + binWidth_obs_x_Region0_1 * L_x_signal_Region0_overallSyst_x_Exp ] = 990/990
RooGaussian::lumiConstraint[ x=Lumi mean=nominalLumi sigma=0.02 ] = 1
RooProdPdf::model_Region0[ lumiConstraint * Region0_model(obs_x_Region0) ] = 990

func

In [19]:
events = 1000
chans = 1
nps = 0
bins = 1
for events in [10,100,1000,10000,100000,1000000,10000000]:
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))



-------------------
Starting to process Region0 channel with 1 observables
lumi str = [1,0,10]
lumi Error str = nominalLumi[1,0,1.2],0.02
model_Region0 has no variation histograms 
processing hist expected0
making normFactor: SignalStrength
signal_Region0 has no variation histograms 
processing hist above_expected0
-----------------------------------------
import model into workspace
RooDataSet::AsimovData[obs_x_Region0,weight:binWeightAsimov] = 1 entries (9 weighted)

RooWorkspace(Region0) Region0 workspace contents

variables
---------
(Lumi,SignalStrength,binWidth_obs_x_Region0_0,binWidth_obs_x_Region0_1,nominalLumi,obs_x_Region0,weightVar)

p.d.f.s
-------
RooRealSumPdf::Region0_model[ binWidth_obs_x_Region0_0 * L_x_model_Region0_overallSyst_x_Exp + binWidth_obs_x_Region0_1 * L_x_signal_Region0_overallSyst_x_Exp ] = 9/9
RooGaussian::lumiConstraint[ x=Lumi mean=nominalLumi sigma=0.02 ] = 1
RooProdPdf::model_Region0[ lumiConstraint * Region0_model(obs_x_Region0) ] = 9

functions
--

In [18]:
events = 1000
chans = 1
nps = 0
bins = 1
for nps in range(10):
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))

this uncertainty is 0
this uncertainty is 0
this uncertainty is 1
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 4
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 4
this uncertainty is 5
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 4
this uncertainty is 5
this uncertainty is 6
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 4
this uncertainty is 5
this uncertainty is 6
this uncertainty is 7
this uncertainty is 0
this uncertainty is 1
this uncertainty is 2
this uncertainty is 3
this uncertainty is 4
this uncertainty is 5
this uncertainty is 6
this uncertainty is 7
this uncertainty is 8


--------